In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
import os 
import json
import spacy
import warnings
import joblib
from tqdm import tqdm
import gensim
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer

warnings.filterwarnings('ignore')
nlp = spacy.load('en_core_web_trf', disable=['parser', 'ner'])
stop = stopwords.words('english')
stemmer = SnowballStemmer('english')

platform_df=pd.read_csv('/workspaces/Crowdfunding-Social-Media-Drivers/Reference Code/BTP - II/Data preperation/data_definition/before_creator.csv')

In [27]:
data=[]
with open(r'/workspaces/Crowdfunding-Social-Media-Drivers/Reference Code/BTP - II/Data preperation/data_definition/creator_data.txt','r', encoding='utf-8') as f:
    for line in f:
        data.append([line.strip()])


In [28]:
creator_df = pd.DataFrame([x[0].split(',') for x in data][1:])

In [29]:
creator_df.loc[332,1] = creator_df.loc[332,2] 
creator_df.loc[332,2] = creator_df.loc[332,3] 
creator_df.loc[646,2] = creator_df.loc[646,2] +','+creator_df.loc[646,3]  

creator_df[creator_df.loc[:,3].notnull()]

,0,1,2,3
332,9 created,109 backed,Benjamin Kepner,Benjamin Kepner
646,First created,0 backed,"William H. Boling, Jr.",Jr.


In [30]:
creator_df.drop(3, inplace=True, axis=1)

In [31]:
creator_df.duplicated().sum()

68

In [32]:
creator_df = creator_df.rename(columns={0:'num_projects',1:'num_backed',2:'verified'})

In [33]:
creator_df['num_projects'] = creator_df.num_projects.str.split(' ').apply(lambda x: x[0])
creator_df['num_projects'] = creator_df.num_projects.str.replace('First', '1')
creator_df['num_projects'] = creator_df.num_projects.astype(int)

In [34]:
creator_df['num_backed'] = creator_df.num_backed.str.strip().str.split(' ').apply(lambda x:x[0]).astype(int)

In [35]:
platform_df.drop('Unnamed: 0', axis=1, inplace=True)

In [36]:
platform_df

,idx,collected,goal_amount,num_backers,category,duration,pledge_types,donate,creator,project_we_love,success,start_month,start_day,population,country
0,0,8.600799,8.496990,66,games,30,7.0,0.0,Janos Stone,1.0,1,6,4,11.973093,MA
1,2,11.626750,10.886128,"1,462",games,30,9.0,0.0,Cherrymochi Game Studio,1.0,1,7,1,15.936166,Japan
2,3,12.960635,10.819778,918,design,30,4.0,0.0,Moment,1.0,1,6,1,13.436372,WA
3,4,9.030855,11.021151,93,food,22,12.0,0.0,Akwasi Brenya-Mensa,1.0,0,6,6,15.837972,UK
4,5,11.866416,9.903488,"1,047",comics,34,8.0,0.0,Adam Lawson,1.0,1,6,1,11.564749,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2463,10.303136,10.126631,640,food,30,14.0,0.0,Ilana Kruger + Kara Cohen,1.0,1,7,1,14.648708,NY
1104,2466,10.922389,10.126631,"1,365",comics,19,8.0,0.0,Lucy Bellwood,1.0,1,7,2,13.357133,OR
1105,2467,9.280985,9.210340,82,journalism,28,10.0,0.0,Elisa Da Prato,1.0,1,7,1,14.648708,NY
1106,2472,10.500372,9.210340,445,food,30,12.0,0.0,Dachi Tea Co.,1.0,1,7,6,12.096297,RI


In [37]:
pd.concat([platform_df,creator_df], axis=1)

,idx,collected,goal_amount,num_backers,category,duration,pledge_types,donate,creator,project_we_love,success,start_month,start_day,population,country,num_projects,num_backed,verified
0,0,8.600799,8.496990,66,games,30,7.0,0.0,Janos Stone,1.0,1,6,4,11.973093,MA,2,10,janos stone
1,2,11.626750,10.886128,"1,462",games,30,9.0,0.0,Cherrymochi Game Studio,1.0,1,7,1,15.936166,Japan,2,0,Maho Williams
2,3,12.960635,10.819778,918,design,30,4.0,0.0,Moment,1.0,1,6,1,13.436372,WA,7,16,Marc Barros
3,4,9.030855,11.021151,93,food,22,12.0,0.0,Akwasi Brenya-Mensa,1.0,0,6,6,15.837972,UK,2,1,Akwasi Brenya-Mensa
4,5,11.866416,9.903488,"1,047",comics,34,8.0,0.0,Adam Lawson,1.0,1,6,1,11.564749,CA,3,88,Adam Lawson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2463,10.303136,10.126631,640,food,30,14.0,0.0,Ilana Kruger + Kara Cohen,1.0,1,7,1,14.648708,NY,1,3,Kara Cohen
1104,2466,10.922389,10.126631,"1,365",comics,19,8.0,0.0,Lucy Bellwood,1.0,1,7,2,13.357133,OR,3,91,Lucy Bellwood
1105,2467,9.280985,9.210340,82,journalism,28,10.0,0.0,Elisa Da Prato,1.0,1,7,1,14.648708,NY,1,0,Elisa Da Prato
1106,2472,10.500372,9.210340,445,food,30,12.0,0.0,Dachi Tea Co.,1.0,1,7,6,12.096297,RI,2,4,Nicholas Palumbo


In [38]:
person = creator_df.verified.str.strip().str.lower() == platform_df.creator.str.strip().str.lower()
person.astype(int)

0       1
1       0
2       0
3       1
4       1
       ..
1103    0
1104    1
1105    1
1106    0
1107    1
Length: 1108, dtype: int64

In [39]:
team = ~person
team

0       False
1        True
2        True
3       False
4       False
        ...  
1103     True
1104    False
1105    False
1106     True
1107    False
Length: 1108, dtype: bool

In [40]:
platform_df['Person'] = person.astype('int8')
platform_df['Team'] = team.astype('int8')

In [41]:
platform_df.drop('creator', axis=1, inplace=True)

In [42]:
final_data = pd.concat([platform_df,creator_df], axis=1)

In [43]:
final_data

,idx,collected,goal_amount,num_backers,category,duration,pledge_types,donate,project_we_love,success,start_month,start_day,population,country,Person,Team,num_projects,num_backed,verified
0,0,8.600799,8.496990,66,games,30,7.0,0.0,1.0,1,6,4,11.973093,MA,1,0,2,10,janos stone
1,2,11.626750,10.886128,"1,462",games,30,9.0,0.0,1.0,1,7,1,15.936166,Japan,0,1,2,0,Maho Williams
2,3,12.960635,10.819778,918,design,30,4.0,0.0,1.0,1,6,1,13.436372,WA,0,1,7,16,Marc Barros
3,4,9.030855,11.021151,93,food,22,12.0,0.0,1.0,0,6,6,15.837972,UK,1,0,2,1,Akwasi Brenya-Mensa
4,5,11.866416,9.903488,"1,047",comics,34,8.0,0.0,1.0,1,6,1,11.564749,CA,1,0,3,88,Adam Lawson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2463,10.303136,10.126631,640,food,30,14.0,0.0,1.0,1,7,1,14.648708,NY,0,1,1,3,Kara Cohen
1104,2466,10.922389,10.126631,"1,365",comics,19,8.0,0.0,1.0,1,7,2,13.357133,OR,1,0,3,91,Lucy Bellwood
1105,2467,9.280985,9.210340,82,journalism,28,10.0,0.0,1.0,1,7,1,14.648708,NY,1,0,1,0,Elisa Da Prato
1106,2472,10.500372,9.210340,445,food,30,12.0,0.0,1.0,1,7,6,12.096297,RI,0,1,2,4,Nicholas Palumbo


## Social Media data - redo

In [44]:
current_dir = '/workspaces/Crowdfunding-Social-Media-Drivers/Data/Kickstarter_only/'

post_data = pd.read_csv(current_dir + '2022-07-22-09-31-10-IST-feed-download.csv')
print(post_data.shape)

post_data = post_data[post_data.loc[:, "Page Name"] == 'Kickstarter'].reset_index()
post_data = post_data.iloc[final_data.idx, :]
post_data['final_data_idx'] = final_data.index.to_list()
print(post_data.shape)
post_data.drop_duplicates(inplace=True, subset=['Message'], keep='last')
print(post_data.shape)

post_data.columns = [x.lower().replace(' ','_') for x in post_data.columns.to_list()]
post_data.drop(['user_name', 'facebook_id', 'page_category', 'page_admin_top_country',
                'overperforming_score_(weighted__—__likes_1x_shares_1x_comments_1x_love_1x_wow_1x_haha_1x_sad_1x_angry_1x_care_1x_)'],
                 axis=1, inplace=True)
 

max_likes, max_comments, max_shares = 541,373,233
post_data = post_data[post_data.likes<max_likes].reset_index(drop=True)
post_data = post_data[post_data.comments<max_comments].reset_index(drop=True)
post_data = post_data[post_data.shares<max_shares].reset_index(drop=True)

#Calculating Age of the post and the page
post_data.page_created = pd.to_datetime(post_data.page_created, dayfirst=True)
post_data.post_created = pd.to_datetime(post_data['post_created'].str.replace('IST', ''))
post_data['current_date'] = pd.to_datetime('2022-07-22 09:31:10') 
post_data['post_age'] = (post_data.current_date - post_data.post_created).dt.days
post_data['page_age'] = (post_data.current_date - post_data.page_created).dt.days

#grouping public engagements into two broad categories
post_data['positive_reactions'] = post_data['love'] + post_data['haha'] + post_data['wow'] + post_data['care']
post_data['negative_reactions'] = post_data['sad'] + post_data['angry']
post_data['emoji_reactions'] = post_data['positive_reactions'] + post_data['negative_reactions'] + post_data['likes']
print(post_data.shape)


(9527, 39)
(1108, 41)
(1108, 41)
(1097, 42)


In [49]:
post_data.rename(columns = {'index':'idx'}, inplace=True)

In [51]:
processed_csv = pd.read_csv("/workspaces/Crowdfunding-Social-Media-Drivers/Data/Original_dataset/processed_data.csv")
processed_csv = processed_csv[processed_csv.page_name == 'Kickstarter']
processed_csv.rename(columns={'index':'idx'}, inplace=True)
final_list = []
for i in post_data.idx.to_list():
    if i in processed_csv.original_index.to_list():
        final_list.append(i)
    
print(len(final_list))

post_data = post_data[post_data.idx.isin(final_list)]
processed_csv = processed_csv[processed_csv.original_index.isin(final_list)]

processed_csv.drop('original_index', axis=1, inplace=True)
processed_csv.reset_index(drop=True, inplace=True)
processed_csv.columns = ['fb_'+x for x in processed_csv.columns.to_list()]

processed_csv


1023


,fb_page_name,fb_likes_at_posting,fb_followers_at_posting,fb_type,fb_likes,fb_comments,fb_shares,fb_post_views,fb_readability,fb_readability_lix,...,fb_topic_1,fb_topic_2,fb_topic_3,fb_topic_4,fb_topic_5,fb_topic_6,fb_topic_7,fb_topic_8,fb_topic_9,fb_topic_10
0,Kickstarter,1589842,1587991,link,1,1,0,2,8.267143,48.595238,...,0.154871,0.000071,0.000029,0.059562,0.161481,0.000033,0.421812,0.000038,0.000027,0.171923
1,Kickstarter,1589866,1588006,link,5,2,0,5,8.475000,33.000000,...,0.000073,0.000096,0.318522,0.380075,0.255895,0.000064,0.000072,0.000067,0.000050,0.000097
2,Kickstarter,1589901,1588019,link,10,2,1,12,4.836818,33.727273,...,0.000036,0.000074,0.242741,0.000100,0.000050,0.450027,0.075764,0.085070,0.000038,0.116083
3,Kickstarter,1590070,1588169,link,7,2,0,9,12.596087,57.782609,...,0.000033,0.473447,0.000087,0.000081,0.000010,0.000079,0.237822,0.000051,0.111665,0.128072
4,Kickstarter,1589930,1588019,link,4,1,0,5,6.933333,29.111111,...,0.000060,0.939369,0.000080,0.000065,0.000032,0.000011,0.000050,0.000082,0.000096,0.000015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,Kickstarter,1535819,1507542,video,102,23,49,31739,37.896842,50.578947,...,0.136253,0.011367,0.386422,0.011367,0.011367,0.261408,0.011367,0.011367,0.011367,0.011367
1019,Kickstarter,1535733,1507458,video,72,1,15,14013,24.544375,44.125000,...,0.139431,0.072760,0.000072,0.072571,0.157052,0.072659,0.000085,0.000036,0.080668,0.180581
1020,Kickstarter,1535733,1507458,video,14,1,2,10616,31.478333,42.333333,...,0.189717,0.011366,0.011369,0.208013,0.011366,0.011367,0.511335,0.011367,0.011366,0.011367
1021,Kickstarter,1535625,1507353,link,16,1,3,17,5.669792,36.583333,...,0.000071,0.171239,0.226855,0.000071,0.000080,0.000081,0.198359,0.147461,0.000074,0.130639


In [52]:
processed_csv.columns

Index(['fb_page_name', 'fb_likes_at_posting', 'fb_followers_at_posting',
       'fb_type', 'fb_likes', 'fb_comments', 'fb_shares', 'fb_post_views',
       'fb_readability', 'fb_readability_lix', 'fb_entropy_scores',
       'fb_perplexity_scores', 'fb_fear', 'fb_anger', 'fb_anticip', 'fb_trust',
       'fb_surprise', 'fb_positive', 'fb_negative', 'fb_sadness', 'fb_disgust',
       'fb_joy', 'fb_entities_identified', 'fb_post_sponsored', 'fb_post_age',
       'fb_page_age', 'fb_positive_reactions', 'fb_negative_reactions',
       'fb_topic_0', 'fb_topic_1', 'fb_topic_2', 'fb_topic_3', 'fb_topic_4',
       'fb_topic_5', 'fb_topic_6', 'fb_topic_7', 'fb_topic_8', 'fb_topic_9',
       'fb_topic_10'],
      dtype='object')

In [53]:
final_data = final_data.iloc[post_data.final_data_idx.values, :].drop('idx', axis=1).reset_index(drop=True)


In [54]:
pd.concat([final_data, processed_csv], axis=1).to_csv('/workspaces/Crowdfunding-Social-Media-Drivers/Data/Kickstarter_only/final_data.csv', index=False)